First, create the model. This must match the model used in the interactive training notebook.

In [ ]:
from jetcam.usb_camera import USBCamera
from torch2trt import TRTModule
from motorsJetson import Motors
from utils import preprocess
import Jetson.GPIO as GPIO
import numpy as np
import torch


# Left motors
ENA = 33
IN1 = 21
IN2 = 22

# Right motors
ENB = 32
IN3 = 26
IN4 = 24

# 50 Hz
FREQUENCY = 50

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('/home/jetson/github/tfg-amariscal/models/road_following_model_trt.pth'))
GPIO.setmode(GPIO.BOARD)
GPIO.setwarnings(False)

motors = Motors(ENA, IN1, IN2, ENB, IN3, IN4, FREQUENCY)

camera = USBCamera(width=224, height=224, capture_width=640, capture_height=480, capture_device=0)

STEERING_GAIN = 0.5
STEERING_BIAS = 0.8

# car.throttle = 0.15

while True:
    image = camera.read()
    print("type: " + str(type(image)))
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    motors.w = x * STEERING_GAIN + STEERING_BIAS
    print("x: " + str(x) + " w: " + str(motors.w))
    motors.mySpeed()

Next, load the saved model.  Enter the model path you used to save.

Create the camera class.

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)